In [1]:
from scipy.misc import imread
from preprocess.normalize import preprocess_signature
import tf_signet
from tf_cnn_model import TF_CNNModel
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import os
import scipy.io
from find_largest_image import find_largest
import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier

c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
os.listdir(os.path.join(os.getcwd(),'data','GPDSSyntheticSignatures4k'))

['data_features.csv', 'firmasSINTESISmanuscritas', 'visual_features.csv']

In [3]:
data_f = pd.read_csv(os.path.join(os.getcwd(),'data','GPDSSyntheticSignatures4k','data_features.csv'))
visual_f = pd.read_csv(os.path.join(os.getcwd(),'data','GPDSSyntheticSignatures4k','visual_features.csv'))

In [4]:
data_f.columns = ['path','user_id','sig_id','fakeness']
data_f.head()

,path,user_id,sig_id,fakeness
0,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,3131,29,True
1,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,3131,7,False
2,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,3131,4,False
3,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,3131,26,True
4,/home/mert/Desktop/sigver_dataset/GPDSSyntheti...,3131,17,False


In [ ]:
visual_f.head()

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.0,0.0,2.086571,1.046970,2.012711,0.0,1.471289,0.0,0.0,1.308222,...,0.000000,2.560242,0.000000,0.000000,1.761090,2.054384,0.0,0.0,1.875244,1.555655
1,0.0,0.0,1.909776,1.295273,0.983222,0.0,0.708483,0.0,0.0,0.912948,...,0.000000,2.821531,0.000000,1.145785,2.461549,1.759992,0.0,0.0,2.124749,1.689826
2,0.0,0.0,1.989003,1.175972,1.191586,0.0,1.047355,0.0,0.0,1.942684,...,0.000000,2.109819,0.017639,0.462309,2.615123,1.609518,0.0,0.0,2.022851,1.887849
3,0.0,0.0,2.345391,1.077277,2.091065,0.0,1.449416,0.0,0.0,1.610568,...,0.000000,2.581949,0.000000,0.000000,1.917314,1.687836,0.0,0.0,2.093908,1.857081
4,0.0,0.0,0.980762,0.578907,0.712328,0.0,0.727942,0.0,0.0,1.355375,...,0.234534,2.123285,0.000000,1.186418,2.821104,1.428768,0.0,0.0,1.720274,1.232280


In [ ]:
print(data_f.shape)
print(visual_f.shape)
print(len(data_f['user_id'].unique()), 'users are enrolled in the dataset')
user_count = len(data_f['user_id'].unique())

(216000, 4)
(216000, 2048)
4000 users are enrolled in the dataset


In [ ]:
user_id_list = np.arange(1,len(data_f['user_id'].unique())+1)
train_idx, test_idx = train_test_split(user_id_list, train_size=0.8, random_state=1923)

c:\users\mert\anaconda3\envs\ml_env\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [ ]:
# Filter train users and test users.
data_f_train = data_f.loc[data_f['user_id'].isin(train_idx)]
data_f_test = data_f.loc[data_f['user_id'].isin(test_idx)]
# Filter fakes and genuines.
data_f_train_gen = data_f_train.loc[data_f_train['fakeness']==False]
data_f_train_fake = data_f_train.loc[data_f_train['fakeness']==True]
data_f_train_gen_5 = data_f_train_gen.loc[data_f_train_gen['sig_id'].isin(np.arange(1,6))]
data_f_train_gen_12 = data_f_train_gen.loc[data_f_train_gen['sig_id'].isin(np.arange(1,13))]

In [ ]:
X_train = visual_f.iloc[data_f_train_gen_12.index]
y_train = data_f_train_gen_12['user_id']

In [ ]:
y_train.head()

1     3131
2     3131
6     3131
18    3131
25    3131
Name: user_id, dtype: int64

In [ ]:
data_f_test_gen = data_f_test.loc[data_f_test['fakeness']==False]
data_f_test_fake = data_f_test.loc[data_f_test['fakeness']==True]
data_f_test_gen_5 = data_f_test_gen.loc[data_f_test_gen['sig_id'].isin(np.arange(1,6))]
data_f_test_gen_12 = data_f_test_gen.loc[data_f_test_gen['sig_id'].isin(np.arange(1,13))]

In [ ]:
X_valid = visual_f.iloc[data_f_test_gen_12.index]
y_valid = data_f_test_gen_12['user_id']

In [ ]:
%%time
clf=XGBClassifier(max_depth=1,tree_method='gpu_hist', verbosity=2, predictor='gpu_predictor')
clf.fit(X_train, y_train)

In [ ]:
'''
clf=LGBMClassifier(device='gpu')
clf.fit(X_train, y_train)
'''

In [ ]:
# TODO Let the fitting end.
# TODO Add benchmark metric above. 
# TODO CNN Model & Training, thundersvm model, gbm model, re-implementation of the paper.